In [1]:
import json, os, sys
%load_ext autoreload
%autoreload 2
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cv2
import tensorflow as tf
from data.loader import load_detection
from utils.session_config import setup_gpus
setup_gpus(True, 0.95)

1 Physical GPUs, 1 Logical GPUs


In [2]:
import numpy as np
from matplotlib import pyplot as plt
from pycocotools.coco import COCO
train_ids = np.load("annotations/train_ids.npy")
test_ids = np.load("annotations/val_ids.npy")
coco = COCO("annotations/integrated_annotation.json")

loading annotations into memory...
Done (t=0.72s)
creating index...
index created!


# 학습데이터 생성 (파일저장) 이미 파일 생성했을 경우 실행 X

In [3]:
# train_ds = load_detection(train_ids, coco, "detection")
# for patches, labels in train_ds.take(1):
#     print(patches.shape)
#     print(labels)

In [4]:
# patch_id = 0
# for patches, labels in train_ds:
#     for patch, label in zip(patches, labels):
#         patch_id+=1
#         patch_array = patch.numpy()
#         label = label.numpy()
#         cv2.imwrite("detection/patches/{:06d}-{}.png".format(patch_id, label), patch_array[:,:,::-1])

In [5]:
# test_ds = load_detection(test_ids, coco, "detection")
# for patches, labels in test_ds.take(1):
#     print(patches.shape)
#     print(labels)

In [6]:
# patch_id = 0
# for patches, labels in test_ds:
#     for patch, label in zip(patches, labels):
#         patch_id+=1
#         patch_array = patch.numpy()
#         label = label.numpy()
#         cv2.imwrite("detection/patches/test/{:06d}-{}.png".format(patch_id, label), patch_array[:,:,::-1])

# data pipeline

In [7]:
split = tf.strings.split

def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  return img

def get_label(file_path):
  # convert the path to a list of path components
  #parts = tf.strings.split(file_path, ["-",".png"])
  # The second to last is the class-directory
  label = int(split(split(file_path, "-")[1], ".png")[0])
  return label

def data_process(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    img = tf.cast(img, tf.float32)
    return img, label

In [8]:
train_ds = tf.data.Dataset.list_files("detection/patches/train/*")
train_ds = train_ds.shuffle(buffer_size=10000, reshuffle_each_iteration=False)
train_ds = train_ds.shuffle(1000).map(data_process)
for images, labels in train_ds.take(1):
    print(images.shape)
    print(labels)

(224, 224, 3)
tf.Tensor(0, shape=(), dtype=int32)


In [9]:
test_ds = tf.data.Dataset.list_files("detection/patches/test/*")
test_ds = test_ds.shuffle(1000).map(data_process)
for images, labels in test_ds.take(1):
    print(images.shape)
    print(labels)

(224, 224, 3)
tf.Tensor(0, shape=(), dtype=int32)


In [10]:
train_ds = train_ds.batch(32).prefetch(tf.data.experimental.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(tf.data.experimental.AUTOTUNE)

In [11]:
for images, labels in train_ds.take(1):
    print(images.shape)
    print(labels)

(32, 224, 224, 3)
tf.Tensor([0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0], shape=(32,), dtype=int32)


# 모델 빌드 및 학습

In [12]:
model = tf.keras.models.load_model("models/detection.h5")

In [13]:
CCE = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
def class_weights_on_CCE(labels, preds):
    loss = CCE(labels, preds)
    class_weight = tf.where(labels > 0, 10.0, 1.0)
    return tf.reduce_mean(loss*class_weight)
    
@tf.function
def train_step(images, labels, model, optimizer, train_loss, train_acc):
    with tf.GradientTape() as tape:
        images = tf.keras.applications.mobilenet_v2.preprocess_input(images)
        preds = model(images, training=True)
        loss = class_weights_on_CCE(labels,preds)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_acc(labels, preds)
    return loss
@tf.function
def test_step(images, labels, model, test_loss, test_acc):
    images = tf.keras.applications.mobilenet_v2.preprocess_input(images)
    preds = model(images, training=False)
    loss = class_weights_on_CCE(labels,preds)
    test_loss(loss)
    test_acc(labels, preds)
    return loss

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


template = "Epoch {} : detection train loss {:5f}, accuracy {:.3f}, val loss {:5f}, val accuracy {:.3f}"

optimizer= tf.keras.optimizers.Adam(learning_rate=1E-4)
EPOCH = 20

log_dir = "logs/detection"
summary_writer = tf.summary.create_file_writer(log_dir)

for epoch in range(1, EPOCH+1): 
    train_loss.reset_states()
    train_acc.reset_states()
    test_loss.reset_states()
    test_acc.reset_states()

    for i, (patches, labels) in enumerate(train_ds):
        loss = train_step(patches, labels, model, optimizer, \
                             train_loss, train_acc)
        print("Epoch : {} train_step : {} loss {}".format(epoch, i, loss))
    
    #print("Epoch : {}".format(train_acc.result().numpy()))
    for i, (patches, labels) in enumerate(test_ds):
        loss = test_step(patches, labels, model, test_loss, test_acc)
        print("Epoch : {} test_step : {} loss {}".format(epoch, i, loss))
    print(template.format(epoch,
                      train_loss.result().numpy(),
                      train_acc.result().numpy(),
                     test_loss.result().numpy(),
                     test_acc.result().numpy()))

    with summary_writer.as_default():
        tf.summary.scalar('train_loss', train_loss.result(), step=epoch)
        tf.summary.scalar('train_acc', train_acc.result().numpy(), step=epoch)
        tf.summary.scalar('test_loss', test_loss.result().numpy(), step=epoch)
        tf.summary.scalar('test_acc', test_acc.result().numpy(), step=epoch)
    model.save("checkpoints/detection/model-{}.h5".format(epoch))

Epoch : 1 train_step : 0 loss 1.2557700872421265
Epoch : 1 train_step : 1 loss 1.768409013748169
Epoch : 1 train_step : 2 loss 0.8349640369415283
Epoch : 1 train_step : 3 loss 0.9307219386100769
Epoch : 1 train_step : 4 loss 0.6780959963798523
Epoch : 1 train_step : 5 loss 0.6969061493873596
Epoch : 1 train_step : 6 loss 0.6439228057861328
Epoch : 1 train_step : 7 loss 0.3394489288330078
Epoch : 1 train_step : 8 loss 0.6408264636993408
Epoch : 1 train_step : 9 loss 0.3299618065357208
Epoch : 1 train_step : 10 loss 0.29225626587867737
Epoch : 1 train_step : 11 loss 0.29940173029899597
Epoch : 1 train_step : 12 loss 0.16543874144554138
Epoch : 1 train_step : 13 loss 0.33421191573143005
Epoch : 1 train_step : 14 loss 0.28710877895355225
Epoch : 1 train_step : 15 loss 0.23311859369277954
Epoch : 1 train_step : 16 loss 0.38644254207611084
Epoch : 1 train_step : 17 loss 0.060135938227176666
Epoch : 1 train_step : 18 loss 0.9124174118041992
Epoch : 1 train_step : 19 loss 0.07163576781749725
E

Epoch : 1 train_step : 161 loss 0.06992606818675995
Epoch : 1 train_step : 162 loss 0.10603301972150803
Epoch : 1 train_step : 163 loss 0.07758985459804535
Epoch : 1 train_step : 164 loss 0.15356135368347168
Epoch : 1 train_step : 165 loss 0.19485339522361755
Epoch : 1 train_step : 166 loss 0.4642188251018524
Epoch : 1 train_step : 167 loss 0.1547860950231552
Epoch : 1 train_step : 168 loss 0.022875532507896423
Epoch : 1 train_step : 169 loss 0.12438929826021194
Epoch : 1 train_step : 170 loss 0.07753923535346985
Epoch : 1 train_step : 171 loss 0.1715589314699173
Epoch : 1 train_step : 172 loss 0.08041395246982574
Epoch : 1 train_step : 173 loss 0.4429000914096832
Epoch : 1 train_step : 174 loss 0.04848792776465416
Epoch : 1 train_step : 175 loss 0.15417706966400146
Epoch : 1 train_step : 176 loss 0.2023467719554901
Epoch : 1 train_step : 177 loss 0.14526036381721497
Epoch : 1 train_step : 178 loss 0.03582043945789337
Epoch : 1 train_step : 179 loss 0.07978653162717819
Epoch : 1 train_

Epoch : 1 train_step : 321 loss 0.353592187166214
Epoch : 1 train_step : 322 loss 0.13538143038749695
Epoch : 1 train_step : 323 loss 0.06367988884449005
Epoch : 1 train_step : 324 loss 0.06835617870092392
Epoch : 1 train_step : 325 loss 0.01644078642129898
Epoch : 1 train_step : 326 loss 0.06339332461357117
Epoch : 1 train_step : 327 loss 0.33568114042282104
Epoch : 1 train_step : 328 loss 0.05097506195306778
Epoch : 1 train_step : 329 loss 0.6701114177703857
Epoch : 1 train_step : 330 loss 0.06259266287088394
Epoch : 1 train_step : 331 loss 0.06241527944803238
Epoch : 1 train_step : 332 loss 0.22165414690971375
Epoch : 1 train_step : 333 loss 0.14288926124572754
Epoch : 1 train_step : 334 loss 0.06962452828884125
Epoch : 1 train_step : 335 loss 0.22340527176856995
Epoch : 1 train_step : 336 loss 0.3150361180305481
Epoch : 1 train_step : 337 loss 0.37813279032707214
Epoch : 1 train_step : 338 loss 0.08620001375675201
Epoch : 1 train_step : 339 loss 0.1246015802025795
Epoch : 1 train_s

Epoch : 1 train_step : 480 loss 0.03809880092740059
Epoch : 1 train_step : 481 loss 0.08321724832057953
Epoch : 1 train_step : 482 loss 0.06432035565376282
Epoch : 1 train_step : 483 loss 0.07559433579444885
Epoch : 1 train_step : 484 loss 0.2708359956741333
Epoch : 1 train_step : 485 loss 0.011483363807201385
Epoch : 1 train_step : 486 loss 0.08508798480033875
Epoch : 1 train_step : 487 loss 0.08089686930179596
Epoch : 1 train_step : 488 loss 0.07930120080709457
Epoch : 1 train_step : 489 loss 0.02019275352358818
Epoch : 1 train_step : 490 loss 0.13631753623485565
Epoch : 1 train_step : 491 loss 0.1274237036705017
Epoch : 1 train_step : 492 loss 0.026390381157398224
Epoch : 1 train_step : 493 loss 0.09457455575466156
Epoch : 1 train_step : 494 loss 0.03710094839334488
Epoch : 1 train_step : 495 loss 0.06604766100645065
Epoch : 1 train_step : 496 loss 0.23349860310554504
Epoch : 1 train_step : 497 loss 0.04394947737455368
Epoch : 1 train_step : 498 loss 0.05376672372221947
Epoch : 1 tr

Epoch : 1 train_step : 639 loss 0.028124932199716568
Epoch : 1 train_step : 640 loss 0.14579804241657257
Epoch : 1 train_step : 641 loss 0.023551490157842636
Epoch : 1 train_step : 642 loss 0.09026038646697998
Epoch : 1 train_step : 643 loss 0.01626165583729744
Epoch : 1 train_step : 644 loss 0.17691220343112946
Epoch : 1 train_step : 645 loss 0.22671189904212952
Epoch : 1 train_step : 646 loss 0.08400142937898636
Epoch : 1 train_step : 647 loss 0.06991039216518402
Epoch : 1 train_step : 648 loss 0.027809133753180504
Epoch : 1 train_step : 649 loss 0.05203789845108986
Epoch : 1 train_step : 650 loss 0.03233852609992027
Epoch : 1 train_step : 651 loss 0.04955676570534706
Epoch : 1 train_step : 652 loss 0.11007604002952576
Epoch : 1 train_step : 653 loss 0.031069085001945496
Epoch : 1 train_step : 654 loss 0.0017131385393440723
Epoch : 1 train_step : 655 loss 0.014912966638803482
Epoch : 1 train_step : 656 loss 0.30865707993507385
Epoch : 1 train_step : 657 loss 0.013705599121749401
Epoc

...

Epoch : 1 train_step : 1267 loss 0.07505740225315094
Epoch : 1 train_step : 1268 loss 0.05505293607711792
Epoch : 1 train_step : 1269 loss 0.09212511777877808
Epoch : 1 train_step : 1270 loss 0.24057507514953613
Epoch : 1 train_step : 1271 loss 0.14619924128055573
Epoch : 1 train_step : 1272 loss 0.2462308704853058
Epoch : 1 train_step : 1273 loss 0.09373483061790466
Epoch : 1 train_step : 1274 loss 1.0626380443572998
Epoch : 1 train_step : 1275 loss 0.17573510110378265
Epoch : 1 train_step : 1276 loss 0.29539796710014343
Epoch : 1 train_step : 1277 loss 0.12281665205955505
Epoch : 1 train_step : 1278 loss 0.15572276711463928
Epoch : 1 train_step : 1279 loss 0.21543052792549133
Epoch : 1 train_step : 1280 loss 0.06194367632269859
Epoch : 1 train_step : 1281 loss 0.09732289612293243
Epoch : 1 train_step : 1282 loss 0.04386662319302559
Epoch : 1 train_step : 1283 loss 0.05199848487973213
Epoch : 1 train_step : 1284 loss 0.017002515494823456
Epoch : 1 train_step : 1285 loss 0.063960224390

Epoch : 1 train_step : 1423 loss 0.08823364973068237
Epoch : 1 train_step : 1424 loss 0.03282708674669266
Epoch : 1 train_step : 1425 loss 0.03582377731800079
Epoch : 1 train_step : 1426 loss 0.192159041762352
Epoch : 1 train_step : 1427 loss 0.046393021941185
Epoch : 1 train_step : 1428 loss 0.026637164875864983
Epoch : 1 train_step : 1429 loss 0.05395595356822014
Epoch : 1 train_step : 1430 loss 0.03560926765203476
Epoch : 1 train_step : 1431 loss 0.005846941377967596
Epoch : 1 train_step : 1432 loss 0.237813800573349
Epoch : 1 train_step : 1433 loss 0.010227270424365997
Epoch : 1 train_step : 1434 loss 0.08959042280912399
Epoch : 1 train_step : 1435 loss 0.14167536795139313
Epoch : 1 train_step : 1436 loss 0.004629668779671192
Epoch : 1 train_step : 1437 loss 0.012524669989943504
Epoch : 1 train_step : 1438 loss 0.035988908261060715
Epoch : 1 train_step : 1439 loss 0.041701167821884155
Epoch : 1 train_step : 1440 loss 0.09300658106803894
Epoch : 1 train_step : 1441 loss 0.0868806689

Epoch : 1 train_step : 1580 loss 0.11132371425628662
Epoch : 1 train_step : 1581 loss 0.05687422305345535
Epoch : 1 train_step : 1582 loss 0.011217039078474045
Epoch : 1 train_step : 1583 loss 0.030240967869758606
Epoch : 1 train_step : 1584 loss 0.10810273140668869
Epoch : 1 train_step : 1585 loss 0.08185477554798126
Epoch : 1 train_step : 1586 loss 0.011033228598535061
Epoch : 1 train_step : 1587 loss 0.004998181946575642
Epoch : 1 train_step : 1588 loss 0.03828493505716324
Epoch : 1 train_step : 1589 loss 0.6459649801254272
Epoch : 1 train_step : 1590 loss 1.2917513847351074
Epoch : 1 train_step : 1591 loss 0.17226532101631165
Epoch : 1 train_step : 1592 loss 0.13934963941574097
Epoch : 1 train_step : 1593 loss 0.029427126049995422
Epoch : 1 train_step : 1594 loss 0.02637207880616188
Epoch : 1 train_step : 1595 loss 0.14092889428138733
Epoch : 1 train_step : 1596 loss 0.5406286120414734
Epoch : 1 train_step : 1597 loss 0.0526638962328434
Epoch : 1 train_step : 1598 loss 0.0545130930

Epoch : 1 train_step : 1736 loss 0.04577355831861496
Epoch : 1 train_step : 1737 loss 0.09199757874011993
Epoch : 1 train_step : 1738 loss 0.10932476073503494
Epoch : 1 train_step : 1739 loss 0.22797971963882446
Epoch : 1 train_step : 1740 loss 0.034271448850631714
Epoch : 1 train_step : 1741 loss 0.05728008225560188
Epoch : 1 train_step : 1742 loss 0.06796219944953918
Epoch : 1 train_step : 1743 loss 0.0055526252835989
Epoch : 1 train_step : 1744 loss 0.14202560484409332
Epoch : 1 train_step : 1745 loss 0.06776107102632523
Epoch : 1 train_step : 1746 loss 0.07005184888839722
Epoch : 1 train_step : 1747 loss 0.46536576747894287
Epoch : 1 train_step : 1748 loss 0.01769455149769783
Epoch : 1 train_step : 1749 loss 0.026656650006771088
Epoch : 1 train_step : 1750 loss 0.012109888717532158
Epoch : 1 train_step : 1751 loss 0.3601658344268799
Epoch : 1 train_step : 1752 loss 0.0330364927649498
Epoch : 1 train_step : 1753 loss 0.005539584904909134
Epoch : 1 train_step : 1754 loss 0.1319837719

Epoch : 1 train_step : 1893 loss 0.05391763895750046
Epoch : 1 train_step : 1894 loss 0.03149048611521721
Epoch : 1 train_step : 1895 loss 0.04974920675158501
Epoch : 1 train_step : 1896 loss 0.04059583321213722
Epoch : 1 train_step : 1897 loss 0.04140577092766762
Epoch : 1 train_step : 1898 loss 0.3720054626464844
Epoch : 1 train_step : 1899 loss 1.1449779272079468
Epoch : 1 train_step : 1900 loss 0.020182933658361435
Epoch : 1 train_step : 1901 loss 0.03671405091881752
Epoch : 1 train_step : 1902 loss 0.03883782774209976
Epoch : 1 train_step : 1903 loss 0.1542557030916214
Epoch : 1 train_step : 1904 loss 0.0546833798289299
Epoch : 1 train_step : 1905 loss 0.06592375040054321
Epoch : 1 train_step : 1906 loss 0.03037431091070175
Epoch : 1 train_step : 1907 loss 0.09019851684570312
Epoch : 1 train_step : 1908 loss 0.03945880010724068
Epoch : 1 train_step : 1909 loss 0.12060848623514175
Epoch : 1 train_step : 1910 loss 0.03352544829249382
Epoch : 1 train_step : 1911 loss 0.03002872690558